In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import pandas as pd
#pip install plotly==5.3.1
from ipynb.fs.full.ploty_zoom import zoom


In [ ]:
def getManualCount(name):
    img = cv2.imread(name,1)
    hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, (0,200,200), (16,255,255))
    cells, _, _, _ = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)
    
    mask = cv2.inRange(hsv, (17,200,200), (76,255,255))
    internal, _, _, _ = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)
    
    mask = cv2.inRange(hsv, (0,0,0), (1,1,255))
    mask = cv2.erode(mask,np.ones((15,15),np.uint8))
    external, _, cc, _ = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)
    
    for c in cc:
        cv2.circle(img, (int(c[0]), int(c[1])), 25, (255,255,255), -1)
    
    return {"method":"Manual",
            "cells":cells-1,
            "internal":internal-1,
            "external":external-1,
            "path":name,
            "time":0}


def getOriginalCount(name):
    name1 = name.replace("samples/","samples/result/") + "_resultText.txt"
    f = open(name1,'r')
    txt = f.read().replace("\x00"," ")
    f.close()
    lines = txt.split("\n")
    data = lines[1].split()
    cells = int(data[1])
    #no txt do proj original esta saindo a contagem errada
    #infectados sao os que estao externos as celulas
    external = int(data[2])
    
    name2 = name.replace("samples/","samples/result/") + "_infoTextHSV.txt"
    f = open(name2,'r')
    txt = f.read().replace("\x00"," ")
    f.close()
    lines = txt.strip().split("\n")
    lines.pop(0)
    internal = len(lines)-external
    
    
    return {"method"  :"Original",
            "cells"   :cells,
            "internal":internal,
            "external":external,
            "time"    :float(data[3]),
            "path"    :name+"_segmented.jpg"}

def getProjectCount(name):
    name = name.replace("samples/","samples/myresults/") + "_result.txt"
    f = open(name,'r')
    txt = f.read().replace("\x00"," ")
    f.close()
    lines = txt.split("\n")
    ret = []
    for i in range(1,len(lines),1):
        data = lines[i].split("\t")
        if len(data) > 1:
            ret.append({
                'method'  : data[0].strip(),
                'cells'   : int(data[1].strip()),
                'internal': int(data[2].strip()),
                'external': int(data[3].strip()),
                'time'    : float(data[4].strip().replace("s","")),
                'path'    : data[5].strip(),
            })
    return ret


def add_to_samples(samples,values):
    if "cells_error" not in values.keys():
        values['cells_error'] = -1
    if "internal_error" not in values.keys():
        values['internal_error'] = -1
    if "external_error" not in values.keys():
        values['external_error'] = -1
        
    samples['method'].append(values["method"])
    samples['cells'].append(values["cells"])
    samples['internal'].append(values["internal"])
    samples['external'].append(values["external"])
    samples['time'].append(values["time"])
    samples['path'].append(values["path"])
    samples['cells_error'].append(values["cells_error"])
    samples['internal_error'].append(values["internal_error"])
    samples['external_error'].append(values["external_error"])
    
def calc_error(manual,dt):
    dt['cells_error']    = abs(manual["cells"]    - dt["cells"])
    dt['internal_error'] = abs(manual["internal"] - dt["internal"])
    dt['external_error'] = abs(manual["external"] - dt["external"])
    return dt


samples = {"sample":[],
           "method":[],
           "cells":[],
           "internal":[],
           "external":[],
           "time":[],
           "path":[],
           "cells_error":[],
           "internal_error":[],
           "external_error":[]}
#for file in glob.glob("./samples/*_count.jpg"):
for i in range(1,10,1):
    file = "./samples/blood_smear_%d_count.jpg" % i
    name = file.replace("_count.jpg","")
    sample_name = name.replace("./samples/","")
    
    samples['sample'].append(sample_name)
    
    manual = getManualCount(file)
    add_to_samples(samples,manual)
    
    data = getProjectCount(name)
    for dt in data:
        samples['sample'].append(sample_name)
        dt = calc_error(manual,dt)
        add_to_samples(samples,dt)
    
    
    orig = getOriginalCount(name)
    samples['sample'].append(sample_name)
    orig = calc_error(manual,orig)
    add_to_samples(samples,orig)

samples = pd.DataFrame(data=samples)
samples

In [ ]:
def graph(smp_name,value):
    smp = samples.query("method != 'Manual' & sample == '%s'"%smp_name)
    ax = smp[["method",value]].plot.bar(x='method',
                                        y=value,
                                        color=['green','orange','red','blue','violet','purple','brown'],
                                        title=smp_name,
                                        figsize=(12,4),
                                        rot=0)
    manual_count = samples.query("method == 'Manual' & sample == '%s'" % smp_name).iloc[0][value]


    ax.axhline(y=manual_count, color='r', linestyle='dotted')
    ax.get_legend().remove()
    plt.legend(labels = ['Manual count'],loc='center right')
    
    if value == 'cells':
        plt.title('Cell count')
    if value == 'internal':
        plt.title('Infected cells')
    if value == 'external':
        plt.title('External infections')
    if value == 'time':
        plt.title('Time (s)')
    
    
graph("blood_smear_1","cells")
graph("blood_smear_1","internal")
graph("blood_smear_1","external")
graph("blood_smear_1","time")

In [ ]:
samples_list = samples[["sample"]].drop_duplicates().values
samples_list = samples_list.reshape(len(samples_list))

fig, axes = plt.subplots(ncols=3,nrows=3,figsize=(15,10))
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle("Cells count error", fontsize=20)
plt.subplots_adjust(hspace = .4)

pos = [(j,i) for j in range(3) for i in range(3)]
i = 0
for smp_name in samples_list:

    smp = samples.query("method != 'Manual' & sample == '%s'" % smp_name)
    smp = pd.concat([smp.query("method != 'Original'").nsmallest(2,'cells_error'),
                     smp.loc[smp['method'] == "Original"]])
    ax = smp[["method","cells_error"]].plot.bar(x='method',
                                                y='cells_error',
                                                color=['green','orange','red'],
                                                ax=axes[pos[i]],
                                                title=smp_name,
                                                rot=0)
    ax.get_legend().remove()
    ax.set_xlabel('')
    i += 1



In [ ]:
samples_list = samples[["sample"]].drop_duplicates().values
samples_list = samples_list.reshape(len(samples_list))

#methods
cols = 6 #1-8 max
#samples
rows = 9


fig, axes = plt.subplots(nrows=rows,ncols=cols,figsize=(30,30))
fig.tight_layout(rect=[0, .03, 0.5, 0.95])
fig.suptitle("Cells count error", fontsize=20, x=0.1)
plt.subplots_adjust( hspace=0.4 )

pos = [(j,i) for j in range(rows) for i in range(cols)]
i = 0
for smp_name in samples_list:

    smp = samples.query("method != 'Manual' & sample == '%s'" % smp_name)
    smp = pd.concat([smp.query("method != 'Original'").nsmallest(cols-2,'cells_error'),
                     smp.loc[smp['method'] == "Original"]])
    ax = smp[["method","cells_error"]].plot.bar(x='method',
                                                y='cells_error',
                                                color=['green','orange','red','blue','purple'],
                                                ax=axes[pos[i]],
                                                title=smp_name,
                                                rot=30)
    ax.get_legend().remove()
    ax.set_xlabel('')
    i += 1
    
    for v in smp.values:
        img = cv2.cvtColor(cv2.imread(v[6]),cv2.COLOR_BGR2RGB)
        axes[pos[i]].set_title(v[1])
        axes[pos[i]].imshow(img)
        axes[pos[i]].axis('off')
        i += 1

plt.savefig("tmpa4.png",dpi=300)
plt.close()
#sometimes it make a cache of image
#change the image name
zoom("tmpa4.png")


# HoughCircles x Original

In [ ]:
smp = samples.query("method in ['HoughCircles','Original']")


k = 0
for v in smp.values:
    if k == 0:
        fig, axes = plt.subplots(ncols=2,figsize=(20,8))
    fig.tight_layout(rect=[0, 0, 1, 0.95])
    fig.suptitle(v[0],fontsize=20)
    img = cv2.cvtColor(cv2.imread(v[6]),cv2.COLOR_BGR2RGB)
    
    axes[k].set_title(v[1]+ " error:%d" % (v[7]))
    axes[k].imshow(img)
    axes[k].axis('off')
    k += 1
    if k == 2:
        plt.show()
        k = 0